In [3]:
import pandas as pd
import yfinance as yf
import numpy as np 
from scipy.stats import gmean

# Start dates - edit as required
start_date = "2023-11-11" # yyyy-mm-dd
end_date = "2024-11-11"

# Load original portfolio
portf = pd.read_csv("portfolios/fin456_portfolio_holdings_t0.csv")

# Load additional portfolio // optional, can be used to compare proposed changes.
# additional_portf = pd.read_csv("fin456_portfolio_holdings_t1.csv")

def calculate_portfolio_metrics(portf):
    # Separate cash and stock rows
    cash_row = portf[portf['TICKER'] == 'USD']
    stock_rows = portf[portf['TICKER'] != 'USD'].copy()  # Ensure this is a copy

    # Fetch data for the tickers in the portfolio
    tickers = stock_rows['TICKER'].tolist()
    data = yf.download(tickers, start=start_date, end=end_date, interval="1mo")['Adj Close']

    # Calculate monthly returns
    returns = data.pct_change().dropna()

    # Calculate total investment in each stock
    current_prices = data.iloc[-1]  # Last row gives the latest prices
    stock_rows['Investment'] = stock_rows['QUANTITY'].values * current_prices.values

    # Check if cash row is empty and handle accordingly
    if not cash_row.empty:
        cash_quantity = cash_row['QUANTITY'].iloc[0]
    else:
        cash_quantity = 0

    # Add the cash row back for total investment
    total_investment = stock_rows['Investment'].sum() + cash_quantity

    # Calculate weights
    stock_rows['Weight'] = stock_rows['Investment'] / total_investment

    # Handle cash weight separately
    cash_weight = cash_quantity / total_investment if cash_quantity > 0 else 0

    # Calculate weighted monthly returns for the portfolio
    weighted_returns = (returns * stock_rows.set_index('TICKER')['Weight']).sum(axis=1)

    # Adjust weighted returns to include cash
    weighted_returns = weighted_returns * (1 - cash_weight)

    # Expected annual return
    # expected_return = weighted_returns.mean() * 12 # using arithmetric mean
    expected_return = (1+weighted_returns).prod()**(12/weighted_returns.size) - 1 # using geometric mean

    # Portfolio variance and standard deviation (risk)
    portfolio_variance = np.dot(stock_rows.set_index('TICKER')['Weight'].T, 
                                np.dot(returns.cov() * 12, 
                                       stock_rows.set_index('TICKER')['Weight']))
    portfolio_variance *= (1 - cash_weight)**2  # Adjust for cash weight
    portfolio_std_dev = np.sqrt(portfolio_variance)

    return expected_return, portfolio_std_dev, weighted_returns, cash_weight

# Function to calculate Beta and Sharpe Ratio
def calculate_beta_sharpe(portfolio_returns, market_returns, risk_free_rate, portfolio_std_dev, expected_return):
    aligned_portfolio_returns, aligned_market_returns = portfolio_returns.align(market_returns, join='inner')

    # Ensure aligned_market_returns is a Series
    if isinstance(aligned_market_returns, pd.DataFrame):
        aligned_market_returns = aligned_market_returns.squeeze()  # Convert single-column DataFrame to Series

    cov_matrix = np.cov(aligned_portfolio_returns, aligned_market_returns)

    # Calculate beta
    beta = cov_matrix[0, 1] / cov_matrix[1, 1]

    # Sharpe Ratio
    sharpe_ratio = (expected_return - risk_free_rate) / portfolio_std_dev
    return beta, sharpe_ratio

# Function to calculate the Alpha
def calculate_alpha(portf_ret, risk_free_rate, beta, market_ret):
    # Alpha = R - Rf - beta (Rm - Rf) ... R is the portf_ret, Rf is the risk_free_rate, beta is the systematic risk of the portfolio, Rm is the market return.
    alpha = portf_ret - risk_free_rate - beta*(market_ret-risk_free_rate)
    return alpha

# Calculate metrics for original portfolio
original_metrics = calculate_portfolio_metrics(portf)

# # Download S&P 500 (or another market index) data
market_data = yf.download('^GSPC', start=start_date, end=end_date, interval="1mo")['Adj Close']

# # Calculate market monthly returns
market_returns = market_data.pct_change().dropna()

# Fetch risk-free rate (10-Year Treasury Yield)
risk_free_data = yf.download('^TNX', start=start_date, end=end_date, interval="1mo")
risk_free_rate = risk_free_data['Adj Close'].dropna().iloc[-1].item() / 100

# Calculate Beta and Sharpe Ratio for both portfolios
original_beta, original_sharpe = calculate_beta_sharpe(
    original_metrics[2], market_returns, risk_free_rate, original_metrics[1], original_metrics[0]
)

# original_alpha = calculate_alpha()

# Calculate metrics for additional portfolio # uncomment for additional portfolio
# additional_metrics = calculate_portfolio_metrics(additional_portf)

# additional_beta, additional_sharpe = calculate_beta_sharpe(  # uncomment for additional portfolio
#     additional_metrics[2], market_returns, risk_free_rate, additional_metrics[1], additional_metrics[0]
# )






ModuleNotFoundError: No module named 'scipy'

In [4]:
original_metrics

NameError: name 'original_metrics' is not defined

In [10]:
aligned_portfolio_returns, aligned_market_returns = original_metrics[2].align(market_returns, join='inner')
aligned_portfolio_returns

Date
2024-01-01    0.006758
2024-02-01    0.037804
2024-03-01    0.036962
2024-04-01   -0.036994
2024-05-01    0.032165
2024-06-01    0.018229
2024-07-01    0.030247
2024-08-01    0.017712
2024-09-01    0.007633
2024-10-01   -0.009142
2024-11-01    0.037091
dtype: float64

In [11]:
aligned_market_returns

Date
2024-01-01    0.015896
2024-02-01    0.051721
2024-03-01    0.031019
2024-04-01   -0.041615
2024-05-01    0.048021
2024-06-01    0.034670
2024-07-01    0.011321
2024-08-01    0.022835
2024-09-01    0.020197
2024-10-01   -0.009897
2024-11-01    0.037075
Name: Adj Close, dtype: float64

In [17]:
# Function to calculate Beta and Sharpe Ratio
def calculate_beta_sharpe(portfolio_returns, market_returns, risk_free_rate, portfolio_std_dev, expected_return):
    aligned_portfolio_returns, aligned_market_returns = portfolio_returns.align(market_returns, join='inner')

    # Ensure aligned_market_returns is a Series
    if isinstance(aligned_market_returns, pd.DataFrame):
        aligned_market_returns = aligned_market_returns.squeeze()  # Convert single-column DataFrame to Series

    cov_matrix = np.cov(aligned_portfolio_returns, aligned_market_returns) # returns cov matrix of [[var(apr), cov(apr,amr)],[cov(amr,apr),var(amr)]]
    print(cov_matrix)
    # Calculate beta
    beta = cov_matrix[0, 1] / cov_matrix[1, 1]

    # Sharpe Ratio
    sharpe_ratio = (expected_return - risk_free_rate) / portfolio_std_dev
    return beta, sharpe_ratio

In [18]:
# Calculate Beta and Sharpe Ratio for both portfolios
original_beta, original_sharpe = calculate_beta_sharpe(
    original_metrics[2], market_returns, risk_free_rate, original_metrics[1], original_metrics[0]
)

# original_alpha = calculate_alpha()

[[0.00054272 0.00057025]
 [0.00057025 0.00072099]]


In [20]:
market_returns

Date
2024-01-01    0.015896
2024-02-01    0.051721
2024-03-01    0.031019
2024-04-01   -0.041615
2024-05-01    0.048021
2024-06-01    0.034670
2024-07-01    0.011321
2024-08-01    0.022835
2024-09-01    0.020197
2024-10-01   -0.009897
2024-11-01    0.037075
Name: Adj Close, dtype: float64

In [19]:
original_beta

0.7909302802270889

In [31]:
original_metrics[2]

Date
2024-01-01    0.006758
2024-02-01    0.037804
2024-03-01    0.036962
2024-04-01   -0.036994
2024-05-01    0.032165
2024-06-01    0.018229
2024-07-01    0.030247
2024-08-01    0.017712
2024-09-01    0.007633
2024-10-01   -0.009142
2024-11-01    0.037091
dtype: float64

In [26]:
def get_exp_ret(ticker, start_date, end_date):

    # Download S&P 500 (or another market index) data
    market_data = yf.download(ticker, start=start_date, end=end_date, interval="1mo")['Adj Close']

    # Calculate market monthly returns
    market_returns = market_data.pct_change().dropna()

    expected_return = (1+market_returns).prod()**(12/market_returns.size) - 1 # using geometric mean

    return expected_return


In [33]:
market_ret = get_exp_ret('^GSPC',start_date,end_date)

[*********************100%%**********************]  1 of 1 completed


In [35]:
original_alpha = calculate_alpha(original_metrics[0],risk_free_rate,original_beta,market_ret) 
original_alpha

-0.015539131756187458

In [1]:
import time
time.time()

1732414386.2541435

In [2]:
time.gmtime(time.time())

time.struct_time(tm_year=2024, tm_mon=11, tm_mday=24, tm_hour=2, tm_min=13, tm_sec=23, tm_wday=6, tm_yday=329, tm_isdst=0)

In [ ]:
a = time.strftime("%Y-%m-%d",time.gmtime(time.time()))

In [10]:
a = time.strptime(a,"%Y-%m-%d")

In [11]:
time.mktime(a)

1732428000.0

In [13]:
a

'2024-11-24'

In [25]:
from datetime import datetime
import pytz

type(datetime)

type

In [20]:
pytz.timezone('America/New_York')

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

In [26]:
datetime.now(pytz.timezone('America/New_York'))

datetime.datetime(2024, 11, 23, 21, 26, 8, 342276, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)

In [28]:
datetime.now(pytz.timezone('America/New_York')).strftime("%Y-%m-%d")

'2024-11-23'

In [30]:
end_date = datetime.now(pytz.timezone('America/New_York')).strftime("%Y-%m-%d") # yyyy-mm-dd
start_date = datetime.now(pytz.timezone('America/New_York')).replace(year=datetime.now(pytz.timezone('America/New_York')).year-1).strftime("%Y-%m-%d")
 

In [31]:
end_date

'2024-11-23'

In [33]:
type(start_date)

str

In [34]:
type("2024")

str